# Import Packages

In [10]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize,Bounds,minimize_scalar
import statsmodels.api as sm  

# Load Data

In [3]:
# load data
path = '/Users/chohasong/Desktop/Python Analytics/Resume Projects/'
hotel_df = pd.read_excel(path + 'yield_management.xlsx',sheet_name='Problem 2')

In [5]:
# remove irrelevant cols
hotel_df = hotel_df.iloc[:,:-2]

In [6]:
# see data 
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   #_adults                      400 non-null    int64  
 1   #_children                    400 non-null    int64  
 2   #_rooms                       400 non-null    int64  
 3   business/pleasure/don't_know  400 non-null    int64  
 4   weekend                       400 non-null    int64  
 5   #_nights                      400 non-null    int64  
 6   time_till_check_in            400 non-null    int64  
 7   past_visitor                  400 non-null    int64  
 8   room_rate                     400 non-null    int64  
 9   channel                       400 non-null    int64  
 10  accept/reject                 400 non-null    int64  
 11  room_rev                      400 non-null    int64  
 12  food_and_bev                  400 non-null    float64
dtypes: fl

# Featre Engineering

In [7]:
# create dummy
hotel_df["channel_1"] = (hotel_df["channel"] == 1).astype(int)
hotel_df["channel_2"] = (hotel_df["channel"] == 2).astype(int)

hotel_df["business"] = (hotel_df["business/pleasure/don't_know"] == 1).astype(int)
hotel_df["pleasure"] = (hotel_df["business/pleasure/don't_know"] == 2).astype(int)


# Modeling: Logistic Regression

How likely is a guest to accept an offer per night room rate? 

In [8]:
y = hotel_df["accept/reject"].astype(int)

X = hotel_df[['#_adults', '#_children', '#_rooms', 'weekend', '#_nights',
           'time_till_check_in', 'past_visitor', 'room_rate',
           "channel_1", "channel_2", "business", "pleasure"]]

# add intercept
X = sm.add_constant(X)

model = sm.Logit(y, X)
result = model.fit(disp=False)
print(result.summary())

                           Logit Regression Results                           
Dep. Variable:          accept/reject   No. Observations:                  400
Model:                          Logit   Df Residuals:                      387
Method:                           MLE   Df Model:                           12
Date:                Wed, 07 Jan 2026   Pseudo R-squ.:                  0.1509
Time:                        16:04:15   Log-Likelihood:                -235.43
converged:                       True   LL-Null:                       -277.25
Covariance Type:            nonrobust   LLR p-value:                 8.272e-13
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  3.4047      0.888      3.835      0.000       1.665       5.145
#_adults               0.0312      0.237      0.132      0.895      -0.434       0.496
#_children          

- For room rate, a $10 increase implies exp(−0.125)=0.883 (about 12% lower odds of acceptance)
- Acceptance odds are lower for phone requests than channel 3, holding other factors fixed
exp(−0.7412)=0.48 (about 52% lower odds)
- Business visit is positive and significant (0.7785), implying higher acceptance odds relative to the omitted purpose baseline exp(0.7785)=2.18 (about 2.2× the odds)

# Modeling: Linear Regression
What is the estimate of auxiliary revenue source from food & beverages?

In [9]:
# create a copy 
hotel_df_b = hotel_df[hotel_df["accept/reject"] == 1].copy()

# create dummies 
hotel_df_b["channel_1"] = (hotel_df_b["channel"] == 1).astype(int)
hotel_df_b["channel_2"] = (hotel_df_b["channel"] == 2).astype(int)

hotel_df_b["business"] = (hotel_df_b["business/pleasure/don't_know"] == 1).astype(int)
hotel_df_b["pleasure"] = (hotel_df_b["business/pleasure/don't_know"] == 2).astype(int)

# define X and y
X = hotel_df_b[['#_adults', '#_children', '#_rooms', 'weekend', '#_nights',
             'time_till_check_in', 'past_visitor', 'room_rate',
             "channel_1", "channel_2", "business", "pleasure"]].copy()

y = hotel_df_b["food_and_bev"]

# add intercept
X = sm.add_constant(X, has_constant="add")

# fit model 
model2 = sm.OLS(y, X)
result2 = model2.fit()

# get summary 
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:           food_and_bev   R-squared:                       0.720
Model:                            OLS   Adj. R-squared:                  0.702
Method:                 Least Squares   F-statistic:                     39.86
Date:                Wed, 07 Jan 2026   Prob (F-statistic):           5.17e-45
Time:                        16:07:33   Log-Likelihood:                -975.18
No. Observations:                 199   AIC:                             1976.
Df Residuals:                     186   BIC:                             2019.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                -46.9382     19

# Revenue Optimization

Optimize room rate per night to maximize total profit

Assume estimated revenue from food & beverage is 40%

In [11]:

logit_params = result.params  # get from logistic reg
fb_params = result2.params     # get from linear reg

def expected_total_profit(price):
    total_profit = 0
    
    for idx in hotel_df.index:
        nights = hotel_df.loc[idx, '#_nights']
        rooms = hotel_df.loc[idx, '#_rooms']
        adults = hotel_df.loc[idx, '#_adults']
        children = hotel_df.loc[idx, '#_children']
        weekend = hotel_df.loc[idx, 'weekend']
        channel_1 = hotel_df.loc[idx, 'channel_1']
        channel_2 = hotel_df.loc[idx, 'channel_2']
        past_visitor = hotel_df.loc[idx, 'past_visitor']
        business = hotel_df.loc[idx, 'business']
        pleasure = hotel_df.loc[idx, 'pleasure']


        
        # P(accept) = exp(b0 + b1*price) / (1 + exp(b0 + b1*price))
        logit_value = logit_params['const'] + logit_params['room_rate'] * price
        prob_accept = np.exp(logit_value) / (1 + np.exp(logit_value))
        
        # room
        room_profit = (price - 50) * nights * rooms
        
        # f&b
        predicted_fb = (fb_params['const'] + 
                       fb_params['room_rate'] * price +
                       fb_params['#_nights'] * nights +
                       fb_params['#_rooms'] * rooms +
                       fb_params['#_adults'] * adults +
                       fb_params['#_children'] * children +
                       fb_params['weekend'] * weekend +
                       fb_params['channel_1'] * channel_1 +
                        fb_params['channel_2'] * channel_2+
                        fb_params['business'] * business+
                        fb_params['pleasure'] * pleasure+
                       fb_params['past_visitor'] * past_visitor)
        
        fb_profit = 0.4 * predicted_fb
        
        expected_profit = prob_accept * (room_profit + fb_profit)
        
        total_profit += expected_profit
    
    return total_profit

# optimization maximize profit (minimize negative profit)
def neg_profit(price):
    return -expected_total_profit(price)

# optimal price
price_min = hotel_df['room_rate'].min()
price_max = hotel_df['room_rate'].max() * 1.5 


result_opt = minimize_scalar(neg_profit, bounds=(price_min, price_max), method='bounded')


print(f'Optimal room rate: ${result_opt.x:.2f} per night per room')
print(f'Maximum expected total profit: ${-result_opt.fun:.2f}')


Optimal room rate: $238.62 per night per room
Maximum expected total profit: $95198.31
